In [1]:
##Estimación de indicadores de agua
#Se borra todo lo que se encuentra en el entorno

#rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr)

Loading required package: pacman



In [2]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

### Indicador de disponibilidad de agua diaria en la vivienda

Para este, se utilizan los datos de la Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH). Dado que esa variable se empezó a capturar en 2020, únicamente se descargan los datos de ese año y 2022.

In [3]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<-"microdatos/enigh"
fin<-"_ns_viviendas_csv.zip"

In [4]:
years<-c("2020", "2022")

In [5]:
# Descargar los archivos, descomprimirlos y renombrarlos
for (i in 1:length(years)) {
  url <- paste0(url_basica, years[i], "/", segunda_parte, years[i], fin)
  destfile <- paste0("data/", years[i], fin)

  # Descargar el archivo
  download.file(url, destfile = destfile)

  # Descomprimir el archivo
  unzip(destfile, exdir = "data")

  # Renombrar el archivo descomprimido a "viviendas_xxxx.csv"
  file.rename(
    from = "data/viviendas.csv",
    to = paste0("data/viviendas_", years[i], ".csv")
  )
}

In [6]:
  #Función de estimación
  
  estimacion <- function(path, variable) {
  # Cargar los datos
  datos <- read_csv(path,show_col_types = FALSE)

  datos <- datos %>%
    mutate(
      across(starts_with("mat") & where(is.character),
             ~ replace_na(parse_number(.x, na = c('', 'NA', '&')), 0)),
    cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")
  
  
  
    # Crear diseño muestral
mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)
    
    
      # Estimar viviendas con agua diaria
  dispagua <- mydesign %>%
group_by(dotac_agua) %>%
    summarise(
      viviendas = survey_total(vartype = "cv"),
      porcentaje = survey_prop(vartype = "cv")
    ) %>%
  #Filtro de viviendas con agua diaria
    filter(dotac_agua == 1) %>%
    mutate(porcentaje = round(porcentaje * 100, 2),
           year = substr(path, 16, 19))
    
    
    
    
    if (variable == "dispagua") {
    return(dispagua)
  }
  else {
    stop("Variable no reconocida")
    }
  }

In [7]:
#Estimar indicador de disponibilidad diaria de agua
tabla_disp_agua<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
  estimacion,
  variable = "dispagua"
)
print("Porcentaje de viviendas con disponibilidad de agua diaria")
tabla_disp_agua

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


[1] "Porcentaje de viviendas con disponibilidad de agua diaria"


dotac_agua,viviendas,viviendas_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,2162481,0.02260787,79.88,0.01928231,2020
1,2300881,0.02524791,78.05,0.02137932,2022


### Indicador de calidad de agua y fugas

Para estos, se utiliza la Encuesta Nacional de Calidad e Impacto Gubernamental (ENCIG). Los indicadores generados son de percepción.

In [8]:
url_encig_basica<-"https://www.inegi.org.mx/contenidos/programas/encig/"
segunda_parte<-"microdatos/encig"

In [9]:
years_encig<-c("2017","2019","2021", "2023")

In [10]:
# Descargar los archivos, descomprimirlos y renombrarlos
for (i in 1:length(years_encig)) {
  # Ajustar el sufijo de la URL y archivo según el año
  if (years_encig[i] %in% c("2017", "2019")) {
    fin <- "_base_datos_dbf.zip"
  } else {
    fin <- "_base_datos_csv.zip"
  }

  # Construir la URL y el nombre del archivo destino
  url <- paste0(url_encig_basica, years_encig[i], "/", segunda_parte, str_sub(years_encig[i], 3, 4), fin)
  print(url)
  destfile <- paste0("data/encig", str_sub(years_encig[i], 3, 4), fin)
  
  # Descargar el archivo
  download.file(url, destfile = destfile)
  
  # Descomprimir el archivo
  unzip(destfile, exdir = "data")
}

[1] "https://www.inegi.org.mx/contenidos/programas/encig/2017/microdatos/encig17_base_datos_dbf.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/encig/2019/microdatos/encig19_base_datos_dbf.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/encig/2021/microdatos/encig21_base_datos_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/encig/2023/microdatos/encig23_base_datos_csv.zip"


In [11]:
# Funcion de estimación
process_files <- function(years, variable) {
  # Función anidada para estimar el indicador`
  estima_indicador <- function(path, variable) {
    # Extraer el año del archivo
    year <- substr(path, 11, 14)
    
    # Leer los datos y hacer las transformaciones necesarias de las variables 
    if (year %in% c("2017", "2019")) {
      datos <- foreign::read.dbf(path) %>%
        janitor::clean_names() %>%
mutate(p4_1_2 = as.numeric(p4_1_2),
    p4_1_4 = as.numeric(p4_1_4),
      fac_p18 = as.numeric(fac_p18),
        upm = as.numeric(upm),
        est_dis = as.numeric(est_dis)) %>%
#Filtrar Ciudad de México
        filter(ent == "09")
    } else if (year %in% c("2021", "2023")) {
      datos <- read_csv(path,show_col_types=FALSE) %>%
        janitor::clean_names() %>%
#Filtrar Ciudad de México
        filter(nom_ent == "CIUDAD DE MEXICO")
    } else {
      stop("Año no soportado.")
    }
    
    # Diseño muestral
    dm <- datos %>%
      as_survey_design(ids = upm, strata = est_dis, weights = fac_p18)
    
    # Calidad del agua
    if (variable == "calidad_agua") {
      calidad_agua <- dm %>%
        group_by(p4_1_2) %>%
        summarise(
          personas = survey_total(vartype = "cv"),
          porcentaje = survey_mean(vartype = "cv")
        ) %>%
#Filtrar calidad mala
        filter(p4_1_2 == 2)%>%
mutate(porcentaje = round(porcentaje * 100, 2),
              year = year)
      return(calidad_agua)

##Atención de fugas de agua

    } else if (variable == "fugas_agua") {
        fugas_agua <- dm %>%
            group_by(p4_1_4) %>%
            summarise(
            personas = survey_total(vartype = "cv"),
            porcentaje = survey_mean(vartype = "cv")
            ) %>%
            filter(p4_1_4 == 1)%>%
            mutate(porcentaje = round(porcentaje * 100, 2),
                     year = year)
        return(fugas_agua)


    } else {
      stop("Variable no reconocida.")
    }
  }
  
  map_dfr(years, function(year) {
    # File path de los archivos
    file_path <- if (year %in% c(2017, 2019)) {
      paste0("data/encig", year, "_01_sec1_3_4_5_8_9_10.dbf")
    } else if (year %in% c(2021, 2023)) {
      paste0("data/encig", year, "_01_sec1_A_3_4_5_8_9_10.csv")
    } else {
      stop("Año no soportado.")
    }
    
    # Print del archivo que se está procesando
    print(paste("Procesando archivo:", file_path))
    
    # Invocar la función de estimación 
    estima_indicador(file_path, variable)
  })
}

In [12]:
#Estimar indicador de calidad de agua
tabla_calidad_agua<-process_files(c(2017,2019,2021,2023), variable = "calidad_agua")
print("Porcentaje de personas de 18 años y más que consideran que el agua de la Ciudad de México es de mala calidad")
tabla_calidad_agua

[1] "Procesando archivo: data/encig2017_01_sec1_3_4_5_8_9_10.dbf"
[1] "Procesando archivo: data/encig2019_01_sec1_3_4_5_8_9_10.dbf"
[1] "Procesando archivo: data/encig2021_01_sec1_A_3_4_5_8_9_10.csv"


New names:
• `` -> `...1`


[1] "Procesando archivo: data/encig2023_01_sec1_A_3_4_5_8_9_10.csv"


New names:
• `` -> `...1`


[1] "Porcentaje de personas de 18 años y más que consideran que el agua de la Ciudad de México es de mala calidad"


p4_1_2,personas,personas_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
2,4402765,0.02835759,50.22,0.02361616,2017
2,3330381,0.02516546,47.99,0.02197730,2019
2,3325619,0.02613323,47.80,0.02211804,2021
2,3540937,0.02558239,46.74,0.02245127,2023


In [13]:
#Estimar indicador de atención de fugas de agua
tabla_fugas_agua<-process_files(c(2017,2019,2021,2023), variable = "fugas_agua")
print("Porcentaje de personas de 18 años y más que consideran que la atención de fugas de agua en la Ciudad de México es rápida")
tabla_fugas_agua

[1] "Procesando archivo: data/encig2017_01_sec1_3_4_5_8_9_10.dbf"
[1] "Procesando archivo: data/encig2019_01_sec1_3_4_5_8_9_10.dbf"
[1] "Procesando archivo: data/encig2021_01_sec1_A_3_4_5_8_9_10.csv"


New names:
• `` -> `...1`


[1] "Procesando archivo: data/encig2023_01_sec1_A_3_4_5_8_9_10.csv"


New names:
• `` -> `...1`


[1] "Porcentaje de personas de 18 años y más que consideran que la atención de fugas de agua en la Ciudad de México es rápida"


p4_1_4,personas,personas_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,2178683,0.03960191,24.85,0.03621243,2017
1,2354994,0.02640651,33.93,0.02473607,2019
1,2250081,0.02684317,32.34,0.02598258,2021
1,2880457,0.02507924,38.02,0.02274877,2023
